In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Multithreaded Architectures

Multithreaded processors are everywhere!!! Take a look of our processor again!

In [2]:
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   48 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              2
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       AuthenticAMD
CPU family:                      25
Model:                           80
Model name:                      AMD Ryzen 7 5700G with Radeon Graphics
Stepping:                        0
Frequency boost:                 enabled
CPU MHz:                         400.000
CPU max MHz:                     4673.0000
CPU min MHz:                     400.0000
BogoMIPS:                        7585.96
Virtualization:                  AMD-V
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                     

In [3]:
! ssh htseng@victini "lscpu"

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          20
On-line CPU(s) list:             0-19
Thread(s) per core:              1
Core(s) per socket:              12
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           151
Model name:                      12th Gen Intel(R) Core(TM) i7-12700K
Stepping:                        2
CPU MHz:                         3600.000
CPU max MHz:                     5000.0000
CPU min MHz:                     800.0000
BogoMIPS:                        7219.20
Virtualization:                  VT-x
L1d cache:                       288 KiB
L1i cache:                       192 KiB
L2 cache:                        7.5 MiB
NUMA node0 CPU(s):             

## Share memory

In [4]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#else","#endif"])])

In [5]:
! make clean; make testloop; make testloop_O3
from IPython.display import IFrame
IFrame("https://hub.escalab.org:8000/user/htseng/terminals/1", width="100%", height="700")

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk *.dump *~ value_of_i vadd_sse vadd_sse_pthread
rm -rf *.dSYM
gcc -O0 -g testloop.c -lpthread -o testloop 
gcc -O3 -g testloop.c -lpthread -o testloop_O3
testloop.c: In function ‘modifyloop’:
testloop.c:23:3: warning: ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
   23 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


In [6]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#ifdef","#else"])])

In [7]:
! make clean; make testloop_volatile; make testloop_O3_volatile
from IPython.display import IFrame
IFrame("https://hub.escalab.org:8000/user/htseng/terminals/1", width="100%", height="700")

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk *.dump *~ value_of_i vadd_sse vadd_sse_pthread
rm -rf *.dSYM
gcc -O0 -g testloop.c -lpthread -o testloop 
gcc -O3 -g testloop.c -lpthread -o testloop_O3
testloop.c: In function ‘modifyloop’:
testloop.c:23:3: warning: ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
   23 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


## Coherency 

Coherency is the protocol maintaining what values should be seen in the system.

Can you read the following code and guess what's the output?

In [8]:
render_code("coherence.c")

// coherence.c:1-34 (34 lines)
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

int loop;

void* modifyloop(void *x)
{
  sleep(1);

  while(loop < 100)
  {
    loop++;
  }
  return NULL;
}

int main()
{
  pthread_t thread;
  loop = 1;
  
  pthread_create(&thread, NULL, modifyloop, NULL);
  while(loop<100)
  {
//    if(loop < 1000)
    fprintf(stderr,"%d ", loop);
  }
  pthread_join(thread, NULL);
  
  fprintf(stderr,"%4d finished\n", loop);
  return 0;
}

Let's run it several times and see what do we get!

In [9]:
! make coherence; ./coherence >& coherence.out; tail -c 1000 coherence.out

make: 'coherence' is up to date.
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [10]:
!./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out; ./coherence >& coherence.out; tail -c 1000 coherence.out

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 90  100 finished


### Share memory example

In [ ]:
compare([do_render_code("vadd_mt.c",show=["CHUNK","#else"]), do_render_code("vadd_mt.c",show=["// interleaving","#endif"])])

In [ ]:
! make vadd_mt_chunk; make vadd_mt

In [ ]:
! echo "size,threads,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "16777216,4," >> stats.csv
! ./vadd_mt_chunk 
! echo -n "16777216,4," >> stats.csv
! ./vadd_mt 

In [ ]:
display_df_mono(render_csv("stats.csv"))

## Consistency

In [ ]:
render_code("fence.c", show=["#ifndef MFENCE","endif"])

In [ ]:
! make clean; make fence

In [ ]:
!rm fence.txt;  touch fence.txt
!for i in {1..3000}; do ./fence 2>> fence.txt; done;

In [ ]:
! grep "(0, 0)" fence.txt |wc
! grep "(0, 1)" fence.txt |wc
! grep "(1, 1)" fence.txt |wc
! grep "(1, 0)" fence.txt |wc

In [ ]:
! rm fence_intel.txt; touch fence_intel.txt
! ssh htseng@victini "cd ~/courses/CS203/demo/multiprocessor; for i in {1..3000}; do ./fence 2>> fence_intel.txt; done;"
! grep "(0, 0)" fence_intel.txt |wc
! grep "(0, 1)" fence_intel.txt |wc
! grep "(1, 1)" fence_intel.txt |wc
! grep "(1, 0)" fence_intel.txt |wc

### "mfence" instructions:

An instruction that forces all updates must finish before making progress after this instruction.

In [ ]:
render_code("fence.c", show=["#ifdef MFENCE", "#endif"])

In [ ]:
!make mfence
!rm mfence.txt; touch mfence.txt
!for i in {1..3000}; do ./fence 2>> mfence.txt; done;
#!ssh htseng@gengar "cd ~/courses/CS203/demo/multiprocessor; for i in {1..3000}; do ./mfence 2>> mfence.txt; done;"

In [ ]:
! grep "(0, 0)" mfence.txt |wc
! grep "(0, 1)" mfence.txt |wc
! grep "(1, 1)" mfence.txt |wc
! grep "(1, 0)" mfence.txt |wc

In [ ]:
! rm mfence_intel.txt; touch mfence_intel.txt
! ssh htseng@victini "cd ~/courses/CS203/demo/multiprocessor; for i in {1..3000}; do ./mfence 2>> mfence_intel.txt; done;"
! grep "(0, 0)" mfence_intel.txt |wc
! grep "(0, 1)" mfence_intel.txt |wc
! grep "(1, 1)" mfence_intel.txt |wc
! grep "(1, 0)" mfence_intel.txt |wc